In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Resize for AlexNet
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
import torch
import torchvision.models as models

# Load Pretrained AlexNet
alexnet = models.alexnet(pretrained=True)

# Modify final layer for binary classification (cats vs dogs)
alexnet.classifier[6] = torch.nn.Linear(4096, 2)

print(alexnet)


In [ ]:
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# Load Data
train_data = datasets.ImageFolder("path/to/train", transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet.to(device)

for epoch in range(10):  
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = alexnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


In [ ]:
pip install ultralytics


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov9.pt')  # Load YOLOv9 Pretrained Model

# Train on dataset
model.train(data="path/to/data.yaml", epochs=20, imgsz=640)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions
y_pred = []
y_true = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = alexnet(images)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Compute Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Example Data
epochs = [1, 2, 3, 4, 5]
f1_scores = [0.78, 0.82, 0.85, 0.88, 0.91]

plt.plot(epochs, f1_scores, marker='o', linestyle='-', color='b')
plt.xlabel("Epochs")
plt.ylabel("F1 Score")
plt.title("F1 Score vs. Epochs")
plt.grid()
plt.show()
